<center><h2><span style="color:peru">WEB SCRAPING - REPOSITORIO DE PUCP-UNALM-UCS</span></h2></center>

Descripción del caso: Se quiere obtener información estructurada de las tesis de pregrado y posgrado de Estadística Informática o de carreras similares desarrolladas a nivel nacional, a través de los repositorios institucionales de las universidades.

Especificaciones:

El repositorio escogido es el de la Universidad Pontificia Universidad Católica del Perú (PUCP), Universidad Científica del Sur y Universidad César Vallejo.
El respectivo url de dicho repositorio es: 

https://repositorio.cientifica.edu.pe/    ------ CIENTIFICA DEL SUR----

https://repositorio.ucv.edu.pe/           -------UCV---------

https://repositorio.pucp.edu.pe/          -------PUCP--------  




### Paso 1: Importamos todas las librerias necesarias



In [1]:
#IMPORTAMOS TODAS LAS librerias que vamos a utilizar 
import pandas as pd
import urllib.request, urllib.parse, urllib.error
import requests
import re
from bs4 import BeautifulSoup
import ssl
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
import time
import math as mt


### Paso 2: Declaramos las siguientes funciones:

total().- Por cada link de repositorio de cada especialidad, extraerá los links de las tesis en la iteración de cada pagina.

distinct().- función en el caso de que exista dos o más items por apartado. Los unirá en una sola string



In [2]:
def total(val,t):# buscar tesis dentro de una área de un repositorio
    html = urllib.request.urlopen(linkss+val+"/recent-submissions").read()
    soup = BeautifulSoup(html, 'html.parser')
    pages = soup.find_all("a", {"class":"next-page-link"})
    string_pages = soup.find_all("p", {"class":"pagination-info"})
    matches = re.findall("\d+", str(string_pages[0]))
    mayor = (max([int(item) for item in matches]) // 20) * 20 #contabiliza el mayor valor de índice de página
    limites = list(range(0, mayor + 20, 20))
    for i in limites:
        html = urllib.request.urlopen(linkss+val+"/recent-submissions?offset={}".format(i)).read()
        soup = BeautifulSoup(html, 'html.parser')
        j = soup.select("a:has(+span.Z3988)")#selecciona el link anterior que contiene esa clase
        for i in j:
            t+=1
            links_tesis["{}{}".format(key,t)] = i.get("href")#guarda el link obtenido    
    
    return t

def distinct(x,lista):#desmembrar lista de listas
    if len(lista) == 0:
        x.append(0)
    elif len(lista) >=2:
        x.append(",".join(lista))
    else:
        x.append(lista[0])

### Paso 3: Colocamos los links de cada grado de inter:

Colocamos los links de cada grado de la misma universidad de la que buscamos extraer. Es el único paso manual dentro de la extracción del presente artículo.



In [3]:
#pucp
#extrae todos los links relacionados a infórmatica y estadística de licenciatura, bachillerato y maestría de la PUCP
url_licenciatura = "https://repositorio.pucp.edu.pe/index/handle/123456789/9119"
url_bachillerato = "https://repositorio.pucp.edu.pe/index/handle/123456789/135254"
url_maestria = "https://repositorio.pucp.edu.pe/index/handle/123456789/9043"
url_total = [url_bachillerato,url_maestria,url_licenciatura]
linkss = "https://repositorio.pucp.edu.pe/"

### Paso 4: Colocamos los links de cada grado de inter:
- Declaramos nuestras listas de almacenaje
- Declaramos las especialidades que tenemos interés.
- Extraemos todos los links mediante la función total() y la almacenamos en el diccionario link_tesis()




In [4]:
links_tesis = dict()#link de tesis por carrera numerado
t = 0
f = 0
mas="?show=full"
uni=[]   #creamos listas para almacenar despues los resultados:
titu=[]
tesis=[]
grad=[]
ases=[]
resu=[]
año=[]
grado = []

for i in url_total:
    links= dict() #links de carreras
    html = urllib.request.urlopen(i).read()
    soup = BeautifulSoup(html, 'html.parser')
    z = soup.find_all("a" , href = True)   
    for i in z:#buscar carreras relacionadas a estadística o infórmatica
        if i.find_all("span", string = re.compile("(?i)(estadística|Ingeniería Informática|Estadística informática|informática|informática .*)|.*Computación?.|.*Sistemas.*?")):
            links[i.text] = i.get("href")
    for key,val in links.items():#itera sobre todos los links de páginas de tesis
        index = total(val,t = f)
        f =index      

### Paso 5: Extraemos la información de cada tesis :
A continuación, se puede utiliza Beautiful Soup para parsear el contenido HTML de cada tesis. Esto permite navegar por el documento de manera sencilla y extraer la información de interés.

Ademas utilizamos Regular Expressions para buscar patrones específicos en el contenido de las paginas de las tesis y extraer información como nombres de autores, títulos, fechas, etc.



In [5]:
for j in links_tesis.values():
        o=linkss+j+mas#Agregamos extensiones para llegar a la pagina donde esta todo lo que queremos hallar 
        a=1              #Usamos un contador            #IMPRIMOS CADA PAGINA
        g=[]                 #guardamos listas auxiliares para añadir los resultados
        h=[]
        html = urllib.request.urlopen(o, context=ctx).read() #leemos el codigo fuente de la pagina
        html = BeautifulSoup(html, 'html.parser')            
        z = html.find_all('td',class_="label-cell")          #leemos todos los td de clase "label-cell"
        for j in range(len(z)): 
                string=z[j]                                  #guardamos cada uno en un string
                string=str(string)
                string = re.sub("</td>","",string)           #lo limpiamos
                string = re.sub('<td class="label-cell">',"",string) #lo limpiamos 
                g.append(string)                             #lo guardamos en la lista g cada una ,ejemplo:"dc.title"
        z=html.find_all('td',class_="word-break")            #leemos todos los td de clase "word-break"
        for j in range(len(z)):                           
                string=z[j]
                string=str(string)                           #guardamos cada uno en un string
                string = re.sub("</td>","",string)           #lo limpiamos
                string = re.sub('<td class="word-break">',"",string) #lo limpiamos
                h.append(string)                             #lo guardamos en la lista g cada una ,ejemplo:"Ingeniera Agroindustrial"
        html=str(html)                                       #convertimos el codigo fuente a str
        for z in range(8):                                   #Creamos un bucle con el rango de los elementos que queremos
            if a==1 and linkss=="https://repositorio.pucp.edu.pe/":   
                try:                                       #si a ==1 
                    n=g.index("dc.date.created")                  #entonces buscara en la lista g el indice donde esta "dc.date.issued"
                    k=h[n]                                       #Buscamos el indice en la lista h, lo guardamos
                    año.append(k)    
                except:
                    año.append(0)                            #Lo agregamos a la lista año
            if a==1 and linkss!="https://repositorio.pucp.edu.pe/":  
                try:                                       #si a ==1 
                    n=g.index("dc.date.issued")                  #entonces buscara en la lista g el indice donde esta "dc.date.issued"
                    k=h[n]                                       #Buscamos el indice en la lista h, lo guardamos
                    año.append(k)    
                except:
                    año.append(0)                         #Lo agregamos a la lista año
            if a==2: 
                try:  
                    n=g.index("dc.title")                        #Hacemos lo mismo que cuando a==1 
                    k=h[n]
                    titu.append(k)
                except:
                    titu.append(0)
            if a==3:  
                try:                                       #Con expresiones regulares encontramos los autores
                    links = re.findall(r'td.class.*dc.contributor.author.*break.>(.*)</td><td>', html)                              #Guardamos la cantidad de autores encontrados
                    distinct(tesis, links)
                except:
                    tesis.append(0)
            if a==4:
                try:
                    n=g.index("thesis.degree.name")              #Hacemos lo mismo que cuando a==1 
                    k=h[n]
                    grad.append(k)
                except:
                    grad.append(0)

            if a==5:                                         #Con expresiones regulares encontramos los asesores
                links = re.findall(r'td.class.*dc.contributor.advisor.*break.>(.*)</td><td>', html)
                distinct(ases, links)                    #Quitamos un elemento de la lista para que no se vuelva a repetir todo
            if a==6:
                try:
                    n=g.index("dc.description.abstract")         #Hacemos lo mismo que cuando a==1 
                    k=h[n]
                    resu.append(k)
                except:
                    resu.append(0)
            if a==7:
                try:
                    n=g.index("dc.publisher")                    #Hacemos lo mismo que cuando a==1 
                    k=h[n]
                    uni.append(k) 
                except:
                    uni.append(0)  
            if a == 8:
                try:
                    linkeado = re.findall(r'td.class.*thesis.degree.level.*break.>(.*)</td><td>', html)
                    distinct(grado, linkeado)
                except:
                    grado.append(0) 

                
            a=a+1
        

In [8]:
len(año)

582

### Paso 6: Almacenaje en dataframe :
Se almacena las listas obtenidas en un dataframe

In [9]:
import pandas as pd
datos = {   
    'Institución':uni,             #Creamos un diccionario con las listas 
    'Titulo': titu,
    'Autor': tesis,
    'Especialidad' : grad,
    'Asesor(es)' : ases,
    'Resumen' : resu,
    "grado": grado,
    "Año":año
}

df = pd.DataFrame(datos)# lo convertimos a un dataframe 
df.to_csv("PUCP_Tesis.csv",sep=";",index=False,encoding="utf-8")
a=pd.read_csv("PUCP_Tesis.csv",sep=";",encoding="utf-8",na_values=["0"])
a

,Institución,Titulo,Autor,Especialidad,Asesor(es),Resumen,grado,Año
0,Pontificia Universidad Católica del Perú,Brecha en ingresos por género en el sector sal...,"Ormeño Meza, Rubén Aaron",Bachiller en Ciencias con mención en Estadística,"Sal y Rosas Celi, Víctor Giancarlo",La brecha salarial entre hombres y mujeres es ...,Bachillerato,2021
1,Pontificia Universidad Católica del Perú,Rediseño de productos de software siguiendo el...,"Salinas Suazo, Elizabeth Karina",Bachiller en Ciencias con mención en Ingenierí...,"Paz Espinoza, Freddy Alberto,Cueva Moscoso, Rony",El presente trabajo de investigación muestra e...,Bachillerato,2020
2,Pontificia Universidad Católica del Perú,Revisión sistemática de las técnicas de Diseño...,"Esparza Cabanillas, Patricia del Pilar",Bachiller en Ciencias con mención en Ingenierí...,"Paz Espinoza, Freddy Alberto,Cueva Moscoso, Rony",Los sistemas de software de punto de venta se ...,Bachillerato,2020
3,Pontificia Universidad Católica del Perú,Predicción temporal de calidad del aire en Lim...,"Paredes Salazar, Diego José",Bachiller en Ciencias con mención en Ingenierí...,"Villanueva Talavera, Edwin Rafael",El presente trabajo explora los estudios en lo...,Bachillerato,2021
4,Pontificia Universidad Católica del Perú,Desarrollo de un modelo de prestación de servi...,"Pérez Cabrera, Estephany Elizabeth",Bachiller en Ciencias con mención en Ingenierí...,"Quispe Vilchez, Eder Ramiro",El presente trabajo de investigación presenta ...,Bachillerato,2020
...,...,...,...,...,...,...,...,...
577,Pontificia Universidad Católica del Perú,"Análisis, diseño y construcción de una herrami...","Camarena Gil, Meylin Cinthia,Pedreschi Núñez, ...",Ingeniero Informático,NaN,En este trabajo de tesis se presenta el desarr...,Título Profesional,2008
578,Pontificia Universidad Católica del Perú,"Análisis, diseño e implementación de un sistem...","Gutiérrez Zúñiga, Jessica,Chanca de la Cruz, M...",Ingeniero Informático,NaN,El presente proyecto de tesis tiene por objeti...,Título Profesional,2009
579,Pontificia Universidad Católica del Perú,Modelado lingüístico-prosódico para un sistema...,"Segura Salas, Elí Rónal",Ingeniero Informático,NaN,El objeto principal de esta tesis ha sido el e...,Título Profesional,2004
580,Pontificia Universidad Católica del Perú,Plataforma Web de apoyo a las PYMES para la ge...,"Rabanal Rabanal, Rosana,Aliaga Rodríguez, Will...",Ingeniero Informático,NaN,El sector de las PYMES es el motor de la econo...,Título Profesional,2006


#### Estadísticas Descriptiva PUCP:

En caso de las especialidades de las tesis, podemos observar que Ingeniería informática es la de mayor representatividad dentro de la PUCP con un 64%

En el caso de los asesores los que tuvieorn mayor participación son Beltrán Castañón, César Armando  con el 9%

In [28]:
a=pd.read_csv("PUCP_Tesis.csv",sep=";",encoding="utf-8",na_values=["0"])# LEEMOS EL ARCHIVO
a
print('PUCP')
#print('Se extrajo',len(ases),'Tesis')
print('\n')
print('------------------------')
print('Años')
print('------------------------')
counts = a['Especialidad'].value_counts(normalize=True).round(2)
print(counts)
counts = counts.apply(lambda x: f"{x:.2%}")
print(counts)
print('------------------------')
print('Asesores')
print('------------------------')
asesors=a['Asesor(es)'].value_counts(normalize=True).round(2)
asesors = asesors.apply(lambda x: f"{x:.2%}")
print(asesors)

PUCP


------------------------
Años
------------------------
Ingeniero Informático                                                0.64
Magíster en Estadística                                              0.10
Magíster en Informática con mención en Ciencias de la Computación    0.09
Magíster en Informática con mención en Ingeniería de Software        0.06
Bachiller en Ciencias con mención en Ingeniería Informática          0.04
Magíster en  Informática con mención en Ingeniería de Software       0.02
Magíster en Informática                                              0.02
Magíster en Ingeniería Informática                                   0.01
Magíster en Ciencias de la Computación                               0.01
Ingeniera Informático                                                0.01
Bachiller en Ciencias con mención en Estadística                     0.00
Magíster en Ingeniería de Software                                   0.00
Ingeniera Informática                             

In [ ]:
#ucv
#extrae todos los links relacionados a infórmatica y estadística de licenciatura, bachillerato y maestría de la PUCP
url_licenciatura = "https://repositorio.ucv.edu.pe/handle/20.500.12692/31"
url_maestria = "https://repositorio.ucv.edu.pe/handle/20.500.12692/66"
url_total = [url_licenciatura,url_maestria]
linkss = "https://repositorio.ucv.edu.pe/"
links_tesis = dict()#link de tesis por carrera numerado
t = 0
f = 0
mas="?show=full"
uni=[]   #creamos listas para almacenar despues los resultados:
titu=[]
tesis=[]
grad=[]
ases=[]
resu=[]
año=[]
grado = []

for i in url_total:
    links= dict() #links de carreras
    html = urllib.request.urlopen(i).read()
    soup = BeautifulSoup(html, 'html.parser')
    z = soup.find_all("a" , href = True)   
    for i in z:#buscar carreras relacionadas a estadística o infórmatica
        if i.find_all("span", string = re.compile("(?i)(estadística|Ingeniería Informática|Estadística informática|informática|informática .*)|.*Computación?.|.*Sistemas.*?")):
            links[i.text] = i.get("href")
    for key,val in links.items():#itera sobre todos los links de páginas de tesis
        index = total(val,t = f)
        f =index  

In [ ]:
for j in links_tesis.values():
        o=linkss+j+mas#Agregamos extensiones para llegar a la pagina donde esta todo lo que queremos hallar 
        a=1              #Usamos un contador             #IMPRIMOS CADA PAGINA
        g=[]                 #guardamos listas auxiliares para añadir los resultados
        h=[]
        html = urllib.request.urlopen(o, context=ctx).read() #leemos el codigo fuente de la pagina
        html = BeautifulSoup(html, 'html.parser')            
        z = html.find_all('td',class_="label-cell")          #leemos todos los td de clase "label-cell"
        for j in range(len(z)): 
                string=z[j]                                  #guardamos cada uno en un string
                string=str(string)
                string = re.sub("</td>","",string)           #lo limpiamos
                string = re.sub('<td class="label-cell">',"",string) #lo limpiamos 
                g.append(string)                             #lo guardamos en la lista g cada una ,ejemplo:"dc.title"
        z=html.find_all('td',class_="word-break")            #leemos todos los td de clase "word-break"
        for j in range(len(z)):                           
                string=z[j]
                string=str(string)                           #guardamos cada uno en un string
                string = re.sub("</td>","",string)           #lo limpiamos
                string = re.sub('<td class="word-break">',"",string) #lo limpiamos
                h.append(string)                             #lo guardamos en la lista g cada una ,ejemplo:"Ingeniera Agroindustrial"
        html=str(html)                                       #convertimos el codigo fuente a str
        for z in range(8):                                   #Creamos un bucle con el rango de los elementos que queremos
            if a==1:   
                try:                                       #si a ==1 
                    n=g.index("dc.date.issued")                  #entonces buscara en la lista g el indice donde esta "dc.date.issued"
                    k=h[n]                                       #Buscamos el indice en la lista h, lo guardamos
                    año.append(k)    
                except:
                    año.append(0)                            #Lo agregamos a la lista año
            if a==2: 
                try:  
                    n=g.index("dc.title")                        #Hacemos lo mismo que cuando a==1 
                    k=h[n]
                    titu.append(k)
                except:
                    titu.append(0)
            if a==3:  
                try:                                       #Con expresiones regulares encontramos los autores
                    links = re.findall(r'td.class.*dc.contributor.author.*break.>(.*)</td><td>', html)                              #Guardamos la cantidad de autores encontrados
                    distinct(tesis, links)
                except:
                    tesis.append(0)
            if a==4:
                try:
                    n=g.index("thesis.degree.name")              #Hacemos lo mismo que cuando a==1 
                    k=h[n]
                    grad.append(k)
                except:
                    grad.append(0)

            if a==5:                                         #Con expresiones regulares encontramos los asesores
                links = re.findall(r'td.class.*dc.contributor.advisor.*break.>(.*)</td><td>', html)
                distinct(ases, links)                    #Quitamos un elemento de la lista para que no se vuelva a repetir todo
            if a==6:
                try:
                    n=g.index("dc.description.abstract")         #Hacemos lo mismo que cuando a==1 
                    k=h[n]
                    resu.append(k)
                except:
                    resu.append(0)
            if a==7:
                try:
                    n=g.index("dc.publisher")                    #Hacemos lo mismo que cuando a==1 
                    k=h[n]
                    uni.append(k) 
                except:
                    uni.append(0)  
            if a == 8:
                try:
                    linkeado = re.findall(r'td.class.*thesis.degree.level.*break.>(.*)</td><td>', html)
                    distinct(grado, linkeado)
                except:
                    grado.append(0) 

                
            a=a+1
        

In [ ]:
import pandas as pd
datos = {   
    'Institución':uni,             #Creamos un diccionario con las listas 
    'Titulo': titu,
    'Autor': tesis,
    'Especialidad' : grad,
    'Asesor(es)' : ases,
    'Resumen' : resu,
    'Año' : año,
    "grado": grado,
}

df = pd.DataFrame(datos)# lo convertimos a un dataframe 
df.to_csv("UCV_Tesis.csv",sep=";",index=False,encoding="utf-8")
a=pd.read_csv("UCV_Tesis.csv",sep=";",encoding="utf-8",na_values=["0"])
a

,Institución,Titulo,Autor,Especialidad,Asesor(es),Resumen,Año,grado
0,Universidad César Vallejo,"Sistema web para la gestión de requerimientos,...","Fuentes Chani, Jonathan Christian Joel",Ingeniero de Sistemas,"Daza Vergaray, Alfredo",Los sistemas de información se han integrado a...,2022,NaN
1,Universidad César Vallejo,Aplicativo móvil para la difusión de informaci...,"Espinoza Quinteros, Carmen Doris,Mendoza Arani...",Ingeniero de Sistemas,"Saavedra Jiménez, Robert Roy",La presente tesis titulada como objetivo princ...,2022,NaN
2,Universidad César Vallejo,Aplicación web para el proceso de ventas en la...,"Huaroc Pari, Jhordy",Ingeniero de Sistemas,"Pérez Farfán, Iván Martin",El presente proyecto detalla la implementación...,2018,NaN
3,Universidad César Vallejo,Aplicación web para mejorar el proceso de coti...,"Herna Hurtado, Jorge Robert,Mateo Camavilca, E...",Ingeniero de Sistemas,"Carranza Barrena, Wilfredo Eduardo",La empresa JCL Suministros E.I.R.L es una empr...,2022,NaN
4,Universidad César Vallejo,Chatbot para la atención de la inscripción en ...,"Bellota Quispe, Joselyn Anais,Mamani Peralta, ...",Ingeniero de Sistemas,"Chumpe Agesto, Juan Brues Lee",La investigación tuvo como problema general ¿C...,2022,NaN
...,...,...,...,...,...,...,...,...
2287,Universidad César Vallejo,Inteligencia de negocios en la gestión del con...,"Espíritu Isidro, Marcelo Leonardo",Maestro en Ingeniería de Sistemas con mención ...,"Flores Sotelo, Willian Sebastian",La presente investigación tuvo como finalidad ...,2018,NaN
2288,Universidad César Vallejo,Sistema de inteligencia de negocios con enfoqu...,"Coronado Loayza, Wilber Edilson",Maestro en Ingeniería de Sistemas con mención ...,"Visurraga Agüero, Joel Martin",La presente tesis se enmarcó dentro de la líne...,2018,NaN
2289,Universidad César Vallejo,Plataforma TEMS Monitor Master en el asegurami...,"Calderón Cáceda, Luis Alonso",Maestro en Ingeniería de Sistemas con mención ...,"Visurraga Agüero, Joel Martin",El presente trabajo de investigación se enmarc...,2017,NaN
2290,Universidad César Vallejo,Software Siletco para la gestión logística de ...,"Flores Roca, Jonathan Pedro",Maestro en Ingeniería de Sistemas con mención ...,"Flores Sotelo, Willian Sebastian",La presente investigación tuvo como finalidad ...,2018,NaN


#### Estadísticas Descriptiva UCV:

En caso de los años de publicación podemos observar el que el año en que se 
publicaron mas tesis fue el 2021 con el 21%

En el caso de los asesores los que tuvieorn mayor participación son Pacheco Torres, Juan Francisco   con el 5%

In [27]:
a=pd.read_csv("UCV_Tesis.csv",sep=";",encoding="utf-8",na_values=["0"])# LEEMOS EL ARCHIVO
a
print('UCV')
#print('Se extrajo',len(ases),'Tesis')
print('\n')
print('------------------------')
print('Años')
print('------------------------')
counts = a['Año'].value_counts(normalize=True).round(2)
print(counts)
counts = counts.apply(lambda x: f"{x:.2%}")
print(counts)
print('------------------------')
print('Asesores')
print('------------------------')
asesors=a['Asesor(es)'].value_counts(normalize=True).round(2)
asesors = asesors.apply(lambda x: f"{x:.2%}")
print(asesors)

UCV


------------------------
Años
------------------------
2021    0.21
2018    0.18
2022    0.14
2020    0.14
2019    0.13
2017    0.09
2016    0.08
2015    0.02
2014    0.01
2013    0.00
2012    0.00
2011    0.00
Name: Año, dtype: float64
2021    21.00%
2018    18.00%
2022    14.00%
2020    14.00%
2019    13.00%
2017     9.00%
2016     8.00%
2015     2.00%
2014     1.00%
2013     0.00%
2012     0.00%
2011     0.00%
Name: Año, dtype: object
------------------------
Asesores
------------------------
Pacheco Torres, Juan Francisco      6.00%
Pérez Farfán, Iván Martín           4.00%
More Valencia, Rubén Alexander      3.00%
Callacná Ponce, Luis Gibson         3.00%
Hilario Falcón, Francisco Manuel    3.00%
                                    ...  
Rivera Crisóstomo, Reneé            0.00%
Rubén Alexander, More Valencia      0.00%
Huarote Zegarra, Raul               0.00%
Galvez Tapia, Orleans Moisés        0.00%
Díaz Reátegui, Monica               0.00%
Name: Asesor(es), Length: 226, 

In [ ]:
#UCS
#extrae todos los links relacionados a infórmatica y estadística de licenciatura, bachillerato y maestría de la PUCP
url_licenciatura = "https://repositorio.cientifica.edu.pe/handle/20.500.12805/252"
url_total = [url_licenciatura]
linkss = "https://repositorio.cientifica.edu.pe/"
links_tesis = dict()#link de tesis por carrera numerado
t = 0
f = 0
mas="?show=full"
uni=[]   #creamos listas para almacenar despues los resultados:
titu=[]
tesis=[]
grad=[]
ases=[]
resu=[]
año=[]
grado = []

for i in url_total:
    links= dict() #links de carreras
    html = urllib.request.urlopen(i).read()
    soup = BeautifulSoup(html, 'html.parser')
    z = soup.find_all("a" , href = True)   
    for i in z:#buscar carreras relacionadas a estadística o infórmatica
        if i.find_all("span", string = re.compile("(?i)(estadística|Ingeniería Informática|Estadística informática|informática|informática .*)|.*Computación?.|.*Sistemas.*?")):
            links[i.text] = i.get("href")
    for key,val in links.items():#itera sobre todos los links de páginas de tesis
        index = total(val,t = f)
        f =index  

In [ ]:
for j in links_tesis.values():
        o=linkss+j+mas#Agregamos extensiones para llegar a la pagina donde esta todo lo que queremos hallar 
        a=1              #Usamos un contador             #IMPRIMOS CADA PAGINA
        g=[]                 #guardamos listas auxiliares para añadir los resultados
        h=[]
        html = urllib.request.urlopen(o, context=ctx).read() #leemos el codigo fuente de la pagina
        html = BeautifulSoup(html, 'html.parser')            
        z = html.find_all('td',class_="label-cell")          #leemos todos los td de clase "label-cell"
        for j in range(len(z)): 
                string=z[j]                                  #guardamos cada uno en un string
                string=str(string)
                string = re.sub("</td>","",string)           #lo limpiamos
                string = re.sub('<td class="label-cell">',"",string) #lo limpiamos 
                g.append(string)                             #lo guardamos en la lista g cada una ,ejemplo:"dc.title"
        z=html.find_all('td',class_="word-break")            #leemos todos los td de clase "word-break"
        for j in range(len(z)):                           
                string=z[j]
                string=str(string)                           #guardamos cada uno en un string
                string = re.sub("</td>","",string)           #lo limpiamos
                string = re.sub('<td class="word-break">',"",string) #lo limpiamos
                h.append(string)                             #lo guardamos en la lista g cada una ,ejemplo:"Ingeniera Agroindustrial"
        html=str(html)                                       #convertimos el codigo fuente a str
        for z in range(8):                                   #Creamos un bucle con el rango de los elementos que queremos
            if a==1:   
                try:                                       #si a ==1 
                    n=g.index("dc.date.issued")                  #entonces buscara en la lista g el indice donde esta "dc.date.issued"
                    k=h[n]                                       #Buscamos el indice en la lista h, lo guardamos
                    año.append(k)    
                except:
                    año.append(0)                            #Lo agregamos a la lista año
            if a==2: 
                try:  
                    n=g.index("dc.title")                        #Hacemos lo mismo que cuando a==1 
                    k=h[n]
                    titu.append(k)
                except:
                    titu.append(0)
            if a==3:  
                try:                                       #Con expresiones regulares encontramos los autores
                    links = re.findall(r'td.class.*dc.contributor.author.*break.>(.*)</td><td>', html)                              #Guardamos la cantidad de autores encontrados
                    distinct(tesis, links)
                except:
                    tesis.append(0)
            if a==4:
                try:
                    n=g.index("thesis.degree.name")              #Hacemos lo mismo que cuando a==1 
                    k=h[n]
                    grad.append(k)
                except:
                    grad.append(0)

            if a==5:                                         #Con expresiones regulares encontramos los asesores
                links = re.findall(r'td.class.*dc.contributor.advisor.*break.>(.*)</td><td>', html)
                distinct(ases, links)                    #Quitamos un elemento de la lista para que no se vuelva a repetir todo
            if a==6:
                try:
                    n=g.index("dc.description.abstract")         #Hacemos lo mismo que cuando a==1 
                    k=h[n]
                    resu.append(k)
                except:
                    resu.append(0)
            if a==7:
                try:
                    n=g.index("dc.publisher")                    #Hacemos lo mismo que cuando a==1 
                    k=h[n]
                    uni.append(k) 
                except:
                    uni.append(0)  
            if a == 8:
                try:
                    linkeado = re.findall(r'td.class.*thesis.degree.level.*break.>(.*)</td><td>', html)
                    distinct(grado, linkeado)
                except:
                    grado.append(0) 

                
            a=a+1

In [ ]:
import pandas as pd
datos = {   
    'Institución':uni,             #Creamos un diccionario con las listas 
    'Titulo': titu,
    'Autor': tesis,
    'Especialidad' : grad,
    'Asesor(es)' : ases,
    'Resumen' : resu,
    'Año' : año,
    "grado": grado,
}

df = pd.DataFrame(datos)# lo convertimos a un dataframe 
df.to_csv("UCS_Tesis.csv",sep=";",index=False,encoding="utf-8")
a=pd.read_csv("UCS_Tesis.csv",sep=";",encoding="utf-8",na_values=["0"])
a

,Institución,Titulo,Autor,Especialidad,Asesor(es),Resumen,Año,grado
0,Universidad Científica del Sur,Implementación del servicio FX Cloud de Amazon...,"Landio Rojas, Raymundo Feliciano,Chipana Avend...",Ingeniero de Sistemas de Información,"Torres Cabanillas, Luis Alberto",El propósito de esta investigación tiene como ...,2022,NaN
1,Universidad Científica del Sur,Procesos robóticos para automatizar el procedi...,"Alarcon Ayquipa, Ester",Ingeniera de Sistemas de Información,"Acosta Medina, Luis Enrique",Los RPA o también conocidos como procesos robó...,2022,NaN
2,Universidad Científica del Sur,Implementación de aplicativo web para mejorar ...,"Mendoza Chiong, Katherine Jannett,Gutierrez Ch...",Ingeniero de Sistemas de Información,"Torres Cabanillas, Luis Alberto","La investigación, dentro del avance de las tec...",2022,NaN
3,Universidad Científica del Sur,Aplicación de aseguramiento de calidad de soft...,"Benites Luna, Carlos Alberto",Ingeniero de Sistemas de Información,"Torres Cabanillas, Luis Alberto",Este estudio tuvo como objetivo determinar de ...,2022,NaN
4,Universidad Científica del Sur,Implementación de una solución de telepresenci...,"Cruz Benites, Walter Jhon,Zamalloa Chiang, Mar...",Ingeniero de Sistemas de Información,"Torres Cabanillas, Luis Alberto",El enfoque de la siguiente investigación fue d...,2022,NaN
...,...,...,...,...,...,...,...,...
145,Universidad Científica del Sur,Planeamiento de un Datamart para Eli Lilly Fil...,"Villar Rivera, John George",Ingeniero de Sistemas,NaN,El presente trabajo tiene por finalidad desarr...,2005,Título profesional
146,Universidad Científica del Sur,Proyecto de implementación de plataforma opera...,"Palacios Mendoza, Guisella Yaninna",Ingeniero de Sistemas,NaN,Este proyecto tiene por finalidad asegurar que...,2005,Título profesional
147,Universidad Científica del Sur,Aceptación de los sistemas de información : un...,"Ontaneda Pérez, Lorena Milagros,Rafael Aliaga,...",Ingeniero de Sistemas,"Bullón Salazar, Luis",En la presente investigación se valida empíric...,2005,Título profesional
148,Universidad Científica del Sur,Gestión de proyectos de sistemas de información,"Luperdi Cruz, Yael Eduardo",Ingeniero de Sistemas,NaN,El presente informe profesional trata sobre un...,2006,Título profesional


### Paso 7: 
#### Estadísticas Descriptiva UCS:

En caso de los años de publicación podemos observar el que el año en que se 
publicaron mas tesis fue el 2015 con el 15%

En el caso de los asesores los que tuvieorn mayor participación son Acosta Medina, Luis Enrique  con el 18%

In [30]:
print('Cientifica del Sur')
print('Se extrajo',len(ases),'Tesis')
print('\n')
print('------------------------')
print('Años')
print('------------------------')
counts = a['Año'].value_counts(normalize=True).round(2)
print(counts)
counts = counts.apply(lambda x: f"{x:.2%}")
print(counts)
print('------------------------')
print('Asesores')
print('------------------------')
asesors=a['Asesor(es)'].value_counts(normalize=True).round(2)
asesors = asesors.apply(lambda x: f"{x:.2%}")
print(asesors)

Cientifica del Sur
Se extrajo 582 Tesis


------------------------
Años
------------------------
2011-05-09    0.09
2015-06-04    0.02
2011-06-13    0.02
2015-05-27    0.01
2015-06-05    0.01
              ... 
2021-12-13    0.00
2022-02-01    0.00
2022-02-06    0.00
2022-02-08    0.00
2021-04-26    0.00
Name: Año, Length: 367, dtype: float64
2011-05-09    9.00%
2015-06-04    2.00%
2011-06-13    2.00%
2015-05-27    1.00%
2015-06-05    1.00%
              ...  
2021-12-13    0.00%
2022-02-01    0.00%
2022-02-06    0.00%
2022-02-08    0.00%
2021-04-26    0.00%
Name: Año, Length: 367, dtype: object
------------------------
Asesores
------------------------
Beltrán Castañón, César Armando                                               9.00%
Cueva Moscoso, Rony                                                           8.00%
Melgar Sasieta, Héctor Andrés                                                 7.00%
Dávila Ramón, Abraham Eliseo                                                  4.00%
Ag